In [1]:
!pip install -U pip
%pip install -Uq "unstructured[all-docs]" Pillow lxml
%pip install -Uq chromadb tiktoken
%pip install -Uq langchain langchain-community langchain-openai
%pip install -Uq python_dotenv
!apt-get update
!apt-get install -y poppler-utils

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 23.7 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradio 5.50.0 requires pillow<12.0,>=8.0, but you have pillow 12.0.0 which is incompatible.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-adk 1.20.0 requires opentelemetry-api<=1.37.0,>=1.37.0, but y

In [7]:
#Need for Colab
!pip install -q --upgrade "Pillow<11"

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pi-heif 1.1.1 requires pillow>=11.1.0, but you have pillow 10.4.0 which is incompatible.


In [3]:
#For Local Run
from dotenv import load_dotenv
load_dotenv()

False

In [4]:
#For Colab Run
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [8]:
from unstructured.partition.pdf import partition_pdf
import os

path = os.path.join(os.getcwd(), "drive/MyDrive/Colab Notebooks/RAG/RRMenu.pdf")

chunks = partition_pdf(
    filename=path,                                                              # only required parameter
    infer_table_structure=False,                                                 # extract tables
    strategy="auto",                                                            # mandatory to infer tables, "high-res" if you extract tables
    languages=["eng"],
    skip_infer_table_types=True,
    # extract_image_block_types=["Image"],                                      # add 'Table' to list to extrac image of table
    # extract_image_block_output_dir= "path",                                   # if none, images and tables will be saved in base64

    # chunking groups related information | Useful for RAG
    chunking_strategy="by_title",                                               # or "basic" | Menu has clear titles
    max_characters=10000,                                                       # max size of chunk
    combine_text_under_n_chars=2000,                                            # comebine different elements when they are under 2000 characters
    new_after_n_chars=6000                                                      # start new part after 6000 characters
    )

In [9]:
print(len(chunks))

9


In [10]:
chunks[0].to_dict()

{'type': 'CompositeElement',
 'element_id': '43c2099fd4e91cc48e55834faccbbcbd',
 'text': 'GOURMET FIRE-GRILLED BURGERS\n\nALL BURGERS INCLUDE A BOTTOMLESS SIDE\n\nSteak Fries • Yukon Chips • Steamed Broccoli • Side Salad Sweet Potato Fries (1.79) • Garlic Fries (1.79)\n\nOUR SIGNATURES Burgers you won’t find anywhere else. 100% Fresh, Never Frozen Beef.\n\nTHE SOUTHERN CHARM BURGER® Brown sugar glaze, candied bacon, Whiskey River® BBQ Sauce, Cheddar, caramelized onions, lettuce and mayo on a toasted brioche bun. 14.99 cal 1190\n\nBLACK & BLEU\n\nSautéed and blackened portobello mushrooms, caramelized onions, creamy cheese sauce, Bleu cheese crumbles, lettuce and roasted garlic aioli on a toasted brioche bun. 15.39 cal 980\n\nWHISKEY RIVER® BBQ\n\nDIP IT. DUNK IT. DOUSE YOUR BURGER.\n\nWhiskey River® BBQ Sauce, crispy onion straws, Cheddar, lettuce, tomatoes and mayo on a toasted sesame bun. 13.79 cal 1140 THE MADLOVE BURGER A Cheddar-and-Parmesan crisp, Provolone, Swiss, jalapeño relis

In [13]:
texts = []

#Incase we decide to extract images/tables in the future
for chunk in chunks:
    if "CompositeElement" in str(type((chunk))):
        texts.append(chunk)

In [14]:
from langchain_core.documents import Document


# Chunks --> Langchain Documents
documents = []
for chunk in texts:
  documents.append(Document(page_content = chunk.text,
                            metadata=chunk.metadata.to_dict()                   # Document() expects metadata as a dict, not unstrucuted metadata
                            )
  )

# print(documents[0])

In [15]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.utils import filter_complex_metadata
from langchain_community.vectorstores import Chroma
import shutil, os

filtered_docs = filter_complex_metadata(documents)                              # langchain doesnt like unstructureds metadata structure

# create embeddings for vector storage
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")               # huggingface is free

# remove chroma_db so you dont re-store data
shutil.rmtree("./chroma_db", ignore_errors=True)
os.makedirs("./chroma_db", exist_ok=True)

# Create vectorstore
vectorstore = Chroma.from_documents(filtered_docs,
                                    embeddings,
                                    persist_directory="./chroma_db"
                                    )

/tmp/ipython-input-4182162611.py:9: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")               # huggingface is free
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [16]:
# get a tuple of the top k chunks and their scores based on their similarity score (i.e. cosine sim)
results = vectorstore.similarity_search_with_relevance_scores("Can i make a custom pizza?", k=3)

# print top k docs and their score
for doc, score in results:
    print(f"Score:  {score} \nText: {doc.page_content} \n {'-'*100}")

Score:  0.20606548467179542 
Text: DESSERTS

MEATS Pepperoni | Shaved Ham | Hardwood-Smoked Bacon Ground Beef | Sausage | Sliced Chicken Breast CHEESES Smoked Provolone VEGGIES, ETC. Mushrooms | Roma Tomatoes | Green Peppers Yellow Onion | Baby Spinach | Banana Peppers | Green Olives Jalapeños | Pineapple

14" pizza has 17 rectangular slices. 12" and 10" pizzas have 14 rectangular slices. ^Calories account for crust, sauce and cheese. **Calorie ranges given for a one-topping pizza. Calories per topping may decrease as the number of toppings per pizza increases. †Gluten free. ††Prepared in a common kitchen alongside items containing gluten.

MOUNTAIN HIGH MUDD PIE

A mountain of chocolate and vanilla ice cream, layered with OREO® cookies, fudge, caramel and whipped cream. 8.79 cal 1340 FRECKLED LEMONADE® CAKE Light and refreshing lemon cake layered with white chocolate–flecked lemon mousse and topped with lemon curd and strawberry purée. 7.99 cal 1060 GOOEY CHOCOLATE BROWNIE CAKE Chocol

/tmp/ipython-input-2579104457.py:2: UserWarning: Relevance scores must be between 0 and 1, got [(Document(metadata={'page_number': 2, 'last_modified': '2025-11-25T01:05:21', 'orig_elements': 'eJzdWW1z4zYO/is8f2o7lqJXS9pveVEc3yZxxnba62x7OxRF2ZqVRR0pxfW299/vISU7yW7aa+cm9yGzM9kQIkAQAIEHyIdfR7ziW163H8t89I6M/IBnPmOZFfp5ZgU8cK0k9F2Lun5Ag5gV2YSPxmS05S3NaUvB8+uICSHzsqYtV2Zd0b3o2o8bXq43LSiu5wW2A67hw67M2w3o8aQnN6KsW8364YM/iexgTBInst2fx+S49pJ+HUS+7b6wNvtBGKm9avlWX+Wu/IVXy4YyPvo3PhRlxT/mpeSsFXKvN5wwUbe4+kkuywd+crO/MP+fi4pm5Fa0PBPikzpZnE5HA39Nt1xzLhY3vO7sJi9G5lb1uqNrc/kPI16vRz8bqmo/bkVeFiU3pvUcL7Rc1/LCleO+c8J3nqu5G3B+rLttxiV2eVrVlv+izTa6SJfLdLFa6m3tvjFHr8q2Mhf60nNJ5HuFm1EroZFvBW4cWDFzYisMvCjJqBcwb/KanvMiTUn8yPa1Y/ql6ziJHZt17NrhC4Se4a157iY9XS3JHW8aLkVdkt/IckMfeE6u6BaLKyrznRC5tdyKT6CeUVyITKXoaiw4LzQD7RQO0L9VJcOm803JPvGanEkOBcn5VZou0yUZRNxJ8SAqUXPyfTqdztLlmKSrc5vcdGojhdgqCFqILSUr/GgF1+up5JDXa6nIj7yqxI7M6xK6/Aadsj1ZNggOtjHLGv+Omw/M8wqGV+TvtKIN/6lznMIV+utdWXPaNBV/Gru3VEragmGlzfRCDOc85m6QJ1aW564VBEVmZZMo

In [ ]:
import transformers, torch, os
from huggingface_hub import login

login(token=os.getenv("HF_TOKEN"))

model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

pipe = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},                               # half the size of weight
    device_map="auto"                                                           # tries gpu, then cpu
)


In [ ]:
query = "Which burgers are vegetarian?"                                         # user input
context = "\n\n".join(doc.page_content for doc, _ in results[:3])

messages = [
    {"role": "system", "content": "Answer only from the menu below. Be brief."},
    {"role": "user", "content": f"Menu:\n{context}\n\n{query}"}
]

outputs = pipe(
    messages,
    max_new_tokens=150,                                                         # number of tokens to generate
    temperature=0.6,                                                            # more likely to select most probable word (logit / 0.7)
    do_sample=True,                                                             # need to set this to true for temperature to have any effect
    top_p=0.9,                                                                  # model choses from top tokens that make up 90% of the probability mass (ignores super low prob tokens from being selected)
    pad_token_id=pipe.tokenizer.eos_token_id
)

# last element is the llm reply
answer = outputs[0]["generated_text"][-1]["content"]
# print(answer)

In [ ]:
import gradio as gr

def chat_interface(message, history):
    response = answer(message)
    history.append((message, response))
    return history, ""   # clear textbox

css = """

@import url('https://fonts.googleapis.com/css2?family=Inter:wght@300;400;500;600&display=swap');

* {
  font-family: "Inter", sans-serif !important;
}

/* Root background and general text color */
#root {
  background-color: #2e3532;
  color: #e0e2db;
  padding: 18px;
  min-height: 100vh;
}

/* Chat container (box that holds messages) */
#chatbot {
  background-color: #e0e2db;
  color: #2e3532;
  border-radius: 12px;
  padding: 20px;
  max-height: 480px;
  overflow-y: auto;
  margin: 0 auto;
  width: 85%;
  box-shadow: 0px 4px 18px rgba(0,0,0,0.15);
}

/* User message bubble (attempt to match Gradio's generated classes) */
#chatbot .message.user,
#chatbot .chatbot-message.user {
  background: #73BA9B;
  color: white;
  border-radius: 12px 12px 0 12px; /* rounded bottom-left square */
  padding: 10px 14px;
  margin: 6px 0;
  max-width: 80%;
  box-shadow: 0 2px 8px rgba(0,0,0,0.2);
}



#chatbot .message, #chatbot .chatbot-message {
  animation: fadeIn 0.25s ease-in-out;
}

@keyframes fadeIn {
  from { opacity: 0; transform: translateY(6px); }
  to { opacity: 1; transform: translateY(0); }
}



/* Assistant/bot message bubble */
#chatbot .message.bot,
#chatbot .chatbot-message.bot {
  background: #ffffff;
  color: #2e3532;
  border-radius: 12px 12px 12px 0; /* rounded bottom-right square */
  padding: 10px 14px;
  margin: 6px 0;
  max-width: 80%;
  box-shadow: 0 2px 8px rgba(0,0,0,0.15);
}

/* Textbox styling */
#msg textarea {
  background-color: #e0e2db !important;
  color: #2e3532 !important;
  border-radius: 8px !important;
  padding: 8px !important;
  box-shadow: none !important;
}

/* Button styling (Clear and default buttons) */
#clear, .gr-button {
  background-color: #01110A !important;
  color: #e0e2db !important;
  border: none !important;
  box-shadow: none !important;
}

/* Title / markdown text color + CENTERING */
#title {
  color: #e0e2db;
  text-align: center;
  width: 100%;
  display: block;
  margin: 0 auto;
}

/* Ensure Gradio's dark borders don't show */
.gradio-container .panel {
  background: transparent !important;
  box-shadow: none !important;
}

#footer {
  text-align: center;
  color: #01110A;
  margin-top: 30px;
  padding-top: 10px;
  font-size: 14px;
  opacity: 0.8;
}
"""

with gr.Blocks(css=css, elem_id="root") as demo:

    gr.Markdown("## 📋 Menu Chatbot", elem_id="title", elem_classes=["title"])


    chatbot = gr.Chatbot(value=[("Assistant", "Hello! How may I assist you today?")], elem_id="chatbot")

    msg = gr.Textbox(
        label="Your question",
        placeholder="Type and press Enter...",
        elem_id="msg"
    )

    examples = gr.Examples(
        examples=[
            "What are some good vegetarian options?",
            "I am not so hungry, what is a good appetizer",
            "If I want to order dessert what are my options?"
        ],
        inputs=[msg]
    )

    clear = gr.Button("Clear", elem_id="clear")

    msg.submit(chat_interface, [msg, chatbot], [chatbot, msg])
    clear.click(lambda: [], None, chatbot)

    gr.Markdown(
        """
        <div id="footer">
            Built for the Resuarant ChatBot RAG Model Project — Penn State · 2025
        </div>
        """,
        elem_id="footer"
    )


with gr.Tabs():
    with gr.Tab("Chat"):
        ...
    with gr.Tab("About"):
        gr.Markdown("""
        ### ℹ️ About This Project
        This chatbot uses a Retrieval-Augmented Generation (RAG) pipeline
        to answer questions about food waste based on EPA reports.
        """)

demo.launch(show_error=True)